# <font color='red' style='font-size: 30px;'> Matriz Origem-Destino</font>
<hr style='border: 2px solid red;'>

## <font color = 'black' style='font-size: 26px;'> Imports </font>  
<hr style = 'border: 1.5px solid black;'>

In [3]:
import pandas as pd
import networkx as nx
import random
import numpy as np
from functions import save_with_pickle, load_with_pickle
AMOSTRAGEM = True

## <font color = 'black' style='font-size: 26px;'> Importando os Dados </font>  
<hr style = 'border: 1.5px solid black;'>

In [5]:
# Importação dos Dados
dataset = pd.read_csv('../Dados/cdr_processed_Uberlândia.csv', sep=',')

In [6]:
# Convertendo as horas paras datetime
dataset['Hora'] = pd.to_datetime(dataset['Hora'], format='%H:%M:%S').dt.hour

In [7]:
# Lendo os IDs encontrados pela camada de relacionamentos
if AMOSTRAGEM:
    nos = load_with_pickle('../Dados/Processados/', 'amostragem_nos')
else:
    nos = load_with_pickle('../Dados/Processados/', 'nos')

In [8]:
# Selecionando esses IDs
dataset = dataset.query("ID_Origem in @nos | ID_Destino in @nos")

## <font color = 'black' style='font-size: 26px;'> Identificando o Local da Residência </font>  
<hr style = 'border: 1.5px solid black;'>

In [10]:
# CDRs referentes ao horário de lazer
weekend = ['2013-03-23', '2013-03-24', '2013-03-30', '2013-03-31', '2013-04-06', '2013-04-07', '2013-04-13', '2013-04-14']
df_non_business = dataset.query("Data in @weekend | (Hora <= 6 | Hora >= 18) ")

In [11]:
# Agrupa por 'ID_Origem' e 'Antena' e conta a frequência de cada combinação
antenna_counts = df_non_business.groupby(['ID_Origem', 'Antena']).size().reset_index(name='Count')

In [12]:
# Encontra a 'Antena' mais utilizada para cada 'ID_Origem'
residence_antenas = antenna_counts.loc[antenna_counts.groupby('ID_Origem')['Count'].idxmax()]
residence_antenas.rename(columns={'Antena' : 'Antena Residência'}, inplace=True)
residence_antenas.drop(columns=['Count'], inplace=True)
residence_antenas.head(3)

,ID_Origem,Antena Residência
0,0000EE5D4A6F13B1261E28A4966BCA5C,55041
1,000104EF060CD662A767A7225AAA768C,25023
7,00010E9AD6FE94A7C878FFBA2EE9FC38,55353


## <font color = 'black' style='font-size: 26px;'> Matriz Origem-Destino </font>  
<hr style = 'border: 1.5px solid black;'>

In [14]:
# Selecionando os dados de segunda a sexta
count_calls_antennas = dataset.query("Data not in @weekend")

In [15]:
# Contagem de ligações feitas em cada antena
count_calls_antennas = dataset.groupby(['ID_Origem', 'Antena']).size().reset_index(name='Count')
count_calls_antennas.drop(columns=['Count'], inplace=True)

In [16]:
count_calls_antennas = pd.merge(count_calls_antennas, residence_antenas, on=['ID_Origem'], how='inner', indicator=False)
count_calls_antennas.head(10)

,ID_Origem,Antena,Antena Residência
0,0000EE5D4A6F13B1261E28A4966BCA5C,55032,55041
1,0000EE5D4A6F13B1261E28A4966BCA5C,55041,55041
2,000104EF060CD662A767A7225AAA768C,25023,25023
3,000104EF060CD662A767A7225AAA768C,55132,25023
4,000104EF060CD662A767A7225AAA768C,55243,25023
5,000104EF060CD662A767A7225AAA768C,56041,25023
6,000104EF060CD662A767A7225AAA768C,56282,25023
7,00010E9AD6FE94A7C878FFBA2EE9FC38,55351,55353
8,00010E9AD6FE94A7C878FFBA2EE9FC38,55353,55353
9,0002448B2A2A138F154E6E90C6D23232,55012,56052


In [17]:
matrix_OD = dict()

for residence in count_calls_antennas['Antena Residência'].unique():
    matrix_OD[residence] = dict()

In [18]:
for _, row in count_calls_antennas.iterrows():
    if not row['Antena'] in matrix_OD[row['Antena Residência']].keys():
        matrix_OD[row['Antena Residência']][row['Antena']] = [row['ID_Origem']]
    else:
        matrix_OD[row['Antena Residência']][row['Antena']].append(row['ID_Origem'])

In [19]:
save_with_pickle(matrix_OD, '../Dados/Processados/', 'matriz_OD')

## <font color = 'black' style='font-size: 26px;'> Criando a Camada </font>  
<hr style = 'border: 1.5px solid black;'>

In [21]:
public_transport_users = list(count_calls_antennas['ID_Origem'].unique())
public_transport_users = random.sample(public_transport_users, int(len(public_transport_users)*0.5))

In [22]:
def cria_arestas(grafo, lista_nos, prob_criacao_arestas, nome_aresta, peso):
    
    # Cria arestas entre todos os nós da lista
    for i in range(len(lista_nos) - 1):
        for j in range(i + 1, len(lista_nos)):
            if random.random() < prob_criacao_arestas:
                if not grafo.has_edge(lista_nos[i], lista_nos[j]):
                    grafo.add_edge(lista_nos[i], lista_nos[j])
                nx.set_edge_attributes(grafo, {(lista_nos[i], lista_nos[j]):{nome_aresta:peso}})
    return grafo

In [23]:
def gera_camada_matrizOD(grafo, nos, matriz_OD, porc_transporte, prob_criacao_arestas, nome_aresta, peso):

    users_transporte_publico = random.sample(users_transporte_publico, int(len(users_transporte_publico)*porc_transporte))
    for key1 in matriz_OD.keys():
        for key2 in matriz_OD[key1].keys():
            elementos_comuns = list(set(users_transporte_publico) & set(matriz_OD[key1][key2]))
            grafo = cria_arestas(grafo, elementos_comuns, prob_criacao_arestas, nome_aresta, peso)
    return grafo